# Pump It Up: Data Mining the Water Table

Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? This is an intermediate-level practice competition. Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

TL:DR: The aim of this assignment if to create a classification model out of the data provided. There are 3 labels ('functional', 'non functional', and 'functional needing repair'). As such we will not make use of logistic regression (even if using one vs rest aproach could be used, using tree classifiers will be simpler) 

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV


# Load our training datasets

In [ ]:
df = pd.read_csv('Training Set values.csv', header = 0)
predictions = pd.read_csv('Training Set labels.csv', header = 0)
predictions = predictions.drop('id', axis=1)

# Find out which columns have NAs

In [ ]:
df.columns[df.isnull().any()].tolist()

Funder refers to who funded the water well, as such NA will be renamed to 'unknown'.

Installer refers to who installed the water well, as such NA will be renamed to 'unknown'.

Subvillage refers to the smallest administrative unit in Tanzania (up to 23k people), we will use 'unknown' for the time being.

Public_meeting unclear reference but is a boolean variable, NAs wil be replaced to False.
Scheme_management' refers to the who operates the well, as such NAs renamed to 'Unknown'.

Scheme_name  refers to the who operates the well, as such NAs renamed to 'Unknown'.

Permit refers to wheter the waterpoint is permitted, since its boolean NAs will be replaced to False.

# Replacing the missing values with 'Unknowns' as explained previously

In [ ]:
df['funder'] =df.funder.fillna("Unknown")
df['installer'] =df.installer.fillna("Unknown")
df['subvillage'] =df.subvillage.fillna("Unknown")
df['scheme_management'] =df.scheme_management.fillna("Unknown")
df['scheme_name'] =df.scheme_name.fillna("Unknown")

# Replacing the missing values with FALSE as explained previously

In [ ]:
df['public_meeting'] =df.public_meeting.fillna(False)
df['permit'] =df.permit.fillna(False)

# Dropping columns
We believe that there are duplicate variables in the dataset. To test this hypothesis we will compare the columns affected. We will proceed to delete the one deemed redundant.

## 1) We count the number of rows

In [ ]:
df.id.count() # There are 59400 rows.

## 2) We compare those columns we believe are identical, if this is the case the sum will be 59400 (each TRUE = 1)

In [ ]:
sum(df['quantity'] == df['quantity_group']) # identical
sum(df['extraction_type'] == df['extraction_type_group']) # 96% idential **** However it seems its the same stuff but with different words (windmill = wind powered)
sum(df['extraction_type'] == df['extraction_type_class']) # 64% identical it seems its a simplified version of the previous one and we will keep this one.
sum(df['payment'] == df['payment_type']) # 64% identical **** However it seems its the same stuff but with different words (on failure = when payment fails), once changed identical
sum(df['water_quality'] == df['quality_group']) # 15% identical **** However it seems its the same stuff but with different words (soft = good), once changed identical
sum(df['source'] == df['source_type']) # 64% identical ****  However it seems its the same stuff but with different words (soft = good), once changed identical
sum(df['quantity'] == df['quantity_group']) # 64% identical ****  However it seems its the same stuff but with different words (machine_dbh = borehole), once changed identical
sum(df['waterpoint_type'] == df['waterpoint_type_group']) #  95%identical, we pick the simplest one aka group

## 3) Time to remove the columns!

In [ ]:
df = df.drop('source', axis=1)
df = df.drop('source_type', axis=1)
df = df.drop('region_code', axis=1)
df = df.drop('waterpoint_type', axis=1)
df = df.drop('quantity', axis=1)
df = df.drop('water_quality', axis=1)
df = df.drop('payment', axis=1)
df = df.drop('extraction_type', axis=1)
df = df.drop('extraction_type_group', axis=1)
df = df.drop('management', axis=1)
df = df.drop('id', axis=1) #no need to keep it!!!
df = df.drop('scheme_name', axis=1) ## We already have the scheme management, which is a more reasonable summary of the data. 
df = df.drop('recorded_by', axis =1) # has one value only, no discriminatory power
df = df.drop('wpt_name', axis =1) # the name of a waterpoint should have no discriminatory value, as such we remove it
df = df.drop('district_code', axis =1) # we have no information to what code does the district correspond, there should be 31 distinct values. However we have 20 values here. We will drop this since such flawed information may become noise


# Checkpoint 1: 
Now that we have filled missing values and deleted duplicate columns we will start working on the dataframe df_semi to further clean the remaining columns to make our data more useful.

In [ ]:
df_semi = df
df_semi.info()


At this moment we will ignore all boolean variables; public_meeting and permit. We we will start with the numerical values, to see if it makes sense transforming it.

# Numerical Variables:


In [ ]:
df_semi.amount_tsh.value_counts() ##
df_semi.gps_height.value_counts() ### height can have 0s and negative numbers. leave as is for now
df_semi.longitude.value_counts() #### highly unlikely to have 0 values here. However changing this value using average location (for example of the basins) could misalocate the water point.
df_semi.latitude.value_counts() # same as lattitude
df_semi.num_private.value_counts()  #### no idea what it could be, no need to change
df_semi.population.value_counts() ### it is fine to have 0s, water points can be in places with no populations
df_semi.construction_year.value_counts()  #### 0s do not make sense, however leave as is as an alternative seems unfeasable

# Categorical Variables:

In [ ]:
df_semi.funder.value_counts() #### has a large amount of distinct values. We will have to engineer here a lot
df_semi.installer.value_counts() #### same issue as funder, we will have to engineer a lot
df_semi.basin.value_counts()  #### no need to work with this data, its perfectly fine as is.
df_semi.subvillage.value_counts() #### 19288 different values, with no simple way to clean this.... we might have to drop this variable given the amount of geographical data we have already
df_semi.region.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.lga.value_counts() #### only 125 variables, might consider dummifying it or label encode it with numerical values
df_semi.ward.value_counts() #### 2092 variables, consider label encode it with numerical values
df_semi.scheme_management.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.extraction_type_class.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.management_group.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.payment_type.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.quality_group.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.quantity_group.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.source_class.value_counts() #### no need to work with this data, its perfectly fine as is.
df_semi.waterpoint_type_group.value_counts() #### no need to work with this data, its perfectly fine as is.



## Funder: 
Has several distinct values. Therefore we will ordering them based on the overall theme. In some cases its due to typos.

In [ ]:
#df_semi.funder.value_counts()
# lets begin with religious organization, we will define a list religious organizations them and replace every instance with 'Religious order'
religious = ["commission", "churc", "mission", "roman", "crist", "catho", "christ", "muslim", "World Vision", "pente", "Tcrs",
             "islam", "kkkt", "dwe", "adra", "wvt", "bsf", "Diocese"]
for x in religious:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'Religious order'

# We see several instances of '0', given  that we already have an umbrela term for unknown values. we will replace '0's for 'Unknown' 
df_semi.loc[df_semi['funder'].str.contains('0', case=False), 'funder'] = 'Unknown'

    
#There are several typos related with 'World Bank', it's their turn to be fixed
world_bank = ["w.b", "wb", "world bank", "w0rld bank", "world_bank", "World"]
for x in world_bank:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'World Bank'  

# Now, to group all the United Nations agencies  funders under 'United Nations', this will also inclue the World Bank
UN = ['Unicef', "Unhcr", "undp", "FAO", "ifad", "imf", "Unesco", "World Bank", "Tasaf", "Unice", "rc", "unis"]
for x in UN:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'United Nations'
    
#For other private initiatives  
private = ['Ikeuchi', "private", "arab comm", "arabs", "bank", "dhv", "Hifab", "gmbh", "wedec", "WU", "Magadini-Makiwaru wa",
           "RWE","Handeni Trunk Main"   ]
for x in private:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'Other private initiatives'
    
#Now its the turn of regional/district/local/village level governmental organizations
district = ["district co", "council", "local council", "local", "dist", "village", "commu", "region", "Rudep", "kidep", "lga",
            "school" ]
for x in district:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'District/local'    
    
# Now, we will group all values related to other countries (including typos) with the value 'Foreign aid'
foreign_aid = ["germany","german","nethe","finland","china","belgian", "british","italy","egypt","iran", "japan", 
               "european union", "swed","korea", "usaid", "usa em", "u.s.a", "holla", "holand", "niger", "irish",
               "swiss","greec", "foreigne", "canada", "kuwai", "Nerthlands", "usa", "eu", "embassy", "norad", "jaica"
               , "African", "danid", "Hesaw", "Jic", "Wua", "snv", "CES"]
for x in foreign_aid:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'Foreign aid'

#decision bodies (ministries and other central governmental institutions) we will fit them under the existing label of "Government Of Tanzania"
central = ["water", "ministry", "Government/tcrs", "Wsdp", "central", "tanzania", "government", "centr", "gov", "Idara ya maji", "tanza"]
for x in central:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'Government Of Tanzania'

# for NGOS
NGO = ["ngos", "habitat","wfp", "wwf", "wfp", "caritas", "cartas" , "internatio","red cross", "redcro","solidarm", "oikos",
       "founda", "club", "acord","kadres", "karadea", "kdrdp","kinapa","mavuno", "drdp", "Oxfam", "Dwsp", "Rwssp", "Amref",
      "lsf", "oxfarm", "Concern World Wide", "Adb", "isf", "shipo", "Plan", "dmdd", "Lvia", "TWESA", "acra", "Sema", "DW", "DH" ]
for x in NGO:
    df_semi.loc[df_semi['funder'].str.contains(x, case=False), 'funder'] = 'NGOs'

#### Given that the majority of terms remaining are very small (in fact most of the 1300s terms are unique values or statsitically insignficiant), these values will have "Others" for now
final_values_funder = ["Government Of Tanzania", "Foreign aid", "NGOs", "Religious order", "District/local", "United Nations", "Unknown", "Other private initiatives" ]    
df_semi.loc[~df_semi["funder"].isin(final_values_funder), "funder"] = "Other"

#df_semi[df_semi['funder'].str.contains("finW", case= False)] #### we used this line to investigate specific values

## Checkpoint 2:
With funder completed, we will save the progress under df_funder, and continue from this point onwards using it.

In [ ]:
df_funder = df_semi

## Installer:
Now we shall proceed to use the same transformations for installer

In [ ]:
# lets begin with religious organization, we will define a list religious organizations them and replace every instance with 'Religious order'
for x in religious:
    df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'Religious order'

# We see several instances of '0', given  that we already have an umbrela term for unknown values. we will replace '0's for 'Unknown' 
df_funder.loc[df_funder['installer'].str.contains('0', case=False), 'installer'] = 'Unknown'
   
#There are several typos related with 'World Bank', it's their turn to be fixed
for x in world_bank:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'World Bank' 
        
# Now, to group all the United Nations agencies  funders under 'United Nations', this will also inclue the World Bank
for x in UN:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'United Nations'
        
######## For other private initiatives ##########   REMEMBER TO IMPROVE 
for x in private:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'Other private initiatives'
    
#Now its the turn of regional/district/local/village level governmental organizations
for x in district:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'District/local'    
    
# Now, we will group all values related to other countries (including typos) with the value 'Foreign aid'
for x in foreign_aid:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'Foreign aid'

    
#For all decision bodies (ministries and other central governmental institutions) we will fit them under the existing label of "Government Of Tanzania"
for x in central:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'Government Of Tanzania'

# for NGOS
for x in NGO:
     df_funder.loc[df_funder['installer'].str.contains(x, case=False), 'installer'] = 'NGOs'
        
#### same as before, the remaining values will have "Others" for now
final_values_installer = ["Government Of Tanzania", "Foreign aid", "NGOs", "Religious order", "District/local", "United Nations", "Unknown", "Other private initiatives" ]    
df_funder.loc[~df_funder["installer"].isin(final_values_installer), "installer"] = "Other"


## Checkpoint 3:
With funder completed, we will save the progress under df_installer , and continue from this point onwards using it.

In [ ]:
df_installer = df_funder #### with installer done, we will utilize df_installer for the next transformation
df_installer = df_installer.drop('subvillage', axis=1) ### we have decided to drop this last variable given that we already have sufficient geographical data, and due to the considerable size of distinct values  (over 19k and no simple way to sumamrise them)

In [ ]:
df_finalv1 = df_installer 
df_finalv1.info()

# We have finished with the training set cleaning, now lets proceed with the testing set cleaning.

## Load the dataset

In [ ]:
dftest = pd.read_csv('Test set values.csv', header = 0)

In [ ]:
ids = pd.DataFrame(dftest['id'])


## Removing Na's

In [ ]:
dftest['funder'] =dftest.funder.fillna("Unknown")
dftest['installer'] =dftest.installer.fillna("Unknown")
dftest['subvillage'] =dftest.subvillage.fillna("Unknown")
dftest['scheme_management'] =dftest.scheme_management.fillna("Unknown")
dftest['scheme_name'] =dftest.scheme_name.fillna("Unknown")
dftest['public_meeting'] =df.public_meeting.fillna(False)
dftest['permit'] =df.permit.fillna(False)


## Dropping columns

In [ ]:
dftest = dftest.drop('source', axis=1)
dftest = dftest.drop('source_type', axis=1)
dftest = dftest.drop('region_code', axis=1)
dftest = dftest.drop('waterpoint_type', axis=1)
dftest = dftest.drop('quantity', axis=1)
dftest = dftest.drop('water_quality', axis=1)
dftest = dftest.drop('payment', axis=1)
dftest = dftest.drop('extraction_type', axis=1)
dftest = dftest.drop('extraction_type_group', axis=1)
dftest = dftest.drop('management', axis=1)
dftest = dftest.drop('id', axis=1)
dftest = dftest.drop('scheme_name', axis=1)  
dftest = dftest.drop('recorded_by', axis =1) 
dftest = dftest.drop('wpt_name', axis =1) 
dftest = dftest.drop('district_code', axis =1) 
dftest = dftest.drop('subvillage', axis=1)

# Feature engineering

## Funder

In [ ]:
#for religions
for x in religious:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'Religious order'

# 0s added to unknown
dftest.loc[dftest['funder'].str.contains('0', case=False), 'funder'] = 'Unknown'
    
#For World Bank
for x in world_bank:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'World Bank'  

# For the UN
for x in UN:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'United Nations'    
    
#For other private initiatives  
for x in private:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'Other private initiatives'
    
#For regional/district/local/village level governmental organizations
for x in district:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'District/local'    
    
# For 'Foreign aid'
for x in foreign_aid:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'Foreign aid'
    
#For central government
for x in central:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'Government Of Tanzania'

# for NGOS
for x in NGO:
    dftest.loc[dftest['funder'].str.contains(x, case=False), 'funder'] = 'NGOs'
# for the rest
dftest.loc[~dftest["funder"].isin(final_values_funder), "funder"] = "Other"


## Installer

In [ ]:
for x in religious:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'Religious order'

# 0s added to unknown
dftest.loc[dftest['installer'].str.contains('0', case=False), 'installer'] = 'Unknown'
    
#TFor World Bank
for x in world_bank:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'World Bank'  

# For the UN
for x in UN:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'United Nations'
    
#For other private initiatives  
for x in private:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'Other private initiatives'
    
#For regional/district/local/village level governmental organizations
for x in district:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'District/local'    
    
# For 'Foreign aid'
for x in foreign_aid:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'Foreign aid'
    
#For central government
for x in central:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'Government Of Tanzania'

# for NGOS
for x in NGO:
    dftest.loc[dftest['installer'].str.contains(x, case=False), 'installer'] = 'NGOs'
# for the rest
dftest.loc[~dftest["installer"].isin(final_values_installer), "installer"] = "Other"





In [ ]:
dftest1 = dftest
dftest1.info()

## Date_recorded: 

Currently the variable is a string. This is not useful, therefor we will separate this variable into 3 new variables 'year', 'month' and 'day' (which will be numericals).

In [ ]:
df_finalv1 = pd.concat([df_finalv1.drop('date_recorded', axis = 1), 
          (df_finalv1.date_recorded.str.split("-|T").str[:3].apply(pd.Series)
          .rename(columns={0:'year', 1:'month', 2:'day'}))], axis = 1)

dftest1 =  pd.concat([dftest1.drop('date_recorded', axis = 1), 
          (dftest1.date_recorded.str.split("-|T").str[:3].apply(pd.Series)
          .rename(columns={0:'year', 1:'month', 2:'day'}))], axis = 1)



In [ ]:
df_finalv1.year = df_finalv1.year.astype(float)
df_finalv1.month = df_finalv1.month.astype(float)
df_finalv1.day = df_finalv1.day.astype(float)

dftest1.year = dftest1.year.astype(float)
dftest1.month = dftest1.month.astype(float)
dftest1.day = dftest1.day.astype(float)

# Base Model
For this first model, we shall use a quick and rough label encoder (using numerical values for both the training and the test

In [ ]:
#Quick and rough label encoder (using the # of classes per variables as the numerical values). We are very much aware that this way has several limiations, as it may give added weight to values with little importance. Nevertheless this for a base model this is perfectly fine as we will improve this model over the next iterations 
for f in df_finalv1.columns:
    if df_finalv1[f].dtype=='object':
        print(f)
        lbl=preprocessing.LabelEncoder()
        lbl.fit(list(df_finalv1[f].values))
        df_finalv1[f]=lbl.transform(list(df_finalv1[f].values))
        
for f in dftest1.columns:
    if dftest1[f].dtype=='object':
        print(f)
        lbl=preprocessing.LabelEncoder()
        lbl.fit(list(dftest1[f].values))
        dftest1[f]=lbl.transform(list(dftest1[f].values))

Below we can see the result of our label encoding.

In [ ]:
df_finalv1.head()
dftest1. head()

We turn our dataframes into numpy arrays, given that sickit learn is better optimised for this format.

In [ ]:
predictions1 = predictions.as_matrix()

## Algorithm 1: Decision Tree Class

In [ ]:
#DTC = DecisionTreeClassifier(max_depth=None, min_samples_split=10,random_state=55)
#scores1 = cross_val_score(DTC, df_finalv1, predictions1.ravel(), cv=5)
#scores1.mean()

## Algorithm 2: Random Forest Classifier

In [ ]:
RFC = RandomForestClassifier(bootstrap=True, n_estimators=700, max_depth=None, min_samples_split=3, max_features='auto',
                             class_weight=None  ,min_samples_leaf= 1, random_state=0, n_jobs = -1)
#scores2 = cross_val_score(RFC, df_finalv1, predictions1.ravel(), cv=5)
#scores2.mean()

## Algorithm 3: Extra Trees Classifier

In [ ]:
#ETC =  ExtraTreesClassifier(bootstrap=True, n_estimators=500, max_depth=None, min_samples_split=2, random_state=0)
#scores3 = cross_val_score(ETC, df_finalv1, predictions1.ravel(), cv=5)
#scores3.mean()

We determine that out of the 3 models, Random forest classifier yields the best results. Hence we will use gridsearch to find the best parameters for our model and at the same time implement cross validation 

# Attempt 1:

In [ ]:
param_grid = {
    'n_estimators': [650, 690 ],
    'max_features': ['auto'], 
    'max_depth': [21]
    
}

model = GridSearchCV(RFC, param_grid, cv=5)
model.fit(df_finalv1, predictions1.ravel()) # the ravel() here was used to transform prediction into a flattened array (a warnign was given if we kept it as a vector)

(Note that this part took trial an error, the parameters above are the results of the tinckering)

In [ ]:
list(df_finalv1)

In [ ]:
model.best_estimator_.feature_importances_

We can see num_private, public_meeting, permit have a very low importance. We will remove them and test again (however we will keep 'year' in spite of the low importance as the effect with 'month' and 'day' is well worth it). 


In [ ]:
df_finalv1 = df_finalv1.drop('num_private', axis=1)
df_finalv1 = df_finalv1.drop('public_meeting', axis=1)
df_finalv1 = df_finalv1.drop('permit', axis=1)

dftest1 = dftest1.drop('num_private', axis=1)
dftest1 = dftest1.drop('public_meeting', axis=1)
dftest1 = dftest1.drop('permit', axis=1)



# Attempt 2:

In [ ]:
param_grid = {
    'n_estimators': [685, 690, 705 ],
    'max_features': ['auto'], 
    'max_depth': [21, 22]
    
}
model = GridSearchCV(RFC, param_grid, n_jobs=8, cv=5)
model.fit(df_finalv1, predictions1.ravel())

In [ ]:
list(df_finalv1)

In [ ]:
model.best_estimator_.feature_importances_

Additional removal of variables based on importance gave worse values. Therefore we will conclude our model here. 

# Time to save and submit:

In [ ]:
results = model.predict(dftest1)

In [ ]:
results =  pd.DataFrame(results, columns=['status_group'])

In [ ]:
results = pd.concat([ids, results], axis=1)

In [ ]:
results.to_csv('prediction33.csv', index = False)

This model yielded 0.8211 score (position #255 out of #4638) in the competition.